In [1]:
from confluent_kafka import Producer
import json
from datetime import datetime

from confluent_kafka import Consumer, KafkaError

topicName ='raw'
topicClean= 'clean-latest'
formatString ='%Y-%m-%dT%H:%M:%S.%f'
bootstrapservers = 'kafka-1:19092,kafka-2:19093'


In [2]:
#%pip install -U avro-python3

In [3]:
customercleanloader = Consumer({
            'bootstrap.servers': bootstrapservers,
            'group.id': 'cleanloader-consumer-group',
            'default.topic.config': {
                'auto.offset.reset': 'earliest'
            }
        })        
cRaw = Consumer({
    'bootstrap.servers': bootstrapservers,
    'group.id': 'raw-to-clean-consumer-group',
    'default.topic.config': {
        'auto.offset.reset': 'largest'
    }
})

In [4]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered Timestamp:{timestamp} to {} [{}] key:{}'.format(msg.topic(), msg.partition(), msg.key(),timestamp=msg.timestamp()))


In [5]:
def getDatetimefromString(datestring):
    datetimeObject = datetime.strptime(datestring.replace('Z','000'), formatString)
    return datetimeObject

In [6]:
def loadCleanConsumer():
    global customercleanloader 
    if(not customercleanloader is None):
        try:
            customercleanloader.close()
        except:
            print("loading new consumer")
    
    customercleanloader = Consumer({
            'bootstrap.servers': bootstrapservers,
            'group.id': 'cleanloader-consumer-group',
            'default.topic.config': {
                'auto.offset.reset': 'earliest'
            }
        })        

    customercleanloader.subscribe([topicClean])

In [7]:
def LoadRawConsumer():
    global cRaw
    if(not cRaw is None):
        try:
            cRaw.close()
        except:
            print("loading new consumer")
    cRaw = Consumer({
        'bootstrap.servers': bootstrapservers,
        'group.id': 'raw-to-clean-consumer-group',
        'default.topic.config': {
            'auto.offset.reset': 'largest'
        }
    })

    cRaw.subscribe([topicName])

In [8]:
lastMeasurementsDict = {
  "boxId": 0}

def SetLastDate(msg):
    keyMsg = msg.key().decode('utf-8')
    measurementarray = json.loads(msg.value().decode('utf-8'))
    valueType= type(measurementarray)
   #if(not valueType =='array'):
   #    print("expected array got {}. ignoring message".format(valueType))
   #    return None
    
    global lastMeasurementsDict 
    global lastRead
    
    for m in measurementarray:
        if not keyMsg in lastMeasurementsDict:
            lastMeasurementsDict[keyMsg]= m
            continue
            
        newMDate = m["createdAt"]
        lastMdate = lastMeasurementsDict[keyMsg]['createdAt']
        if( getDatetimefromString(lastMdate) <  getDatetimefromString(newMDate)) :
            lastMeasurementsDict[keyMsg] = m          
    
def loadCleanData():
    """
    loads the latest measurement time from the clean topic
    """
    go_on = True
    stopafter =5
    counter =0
    while go_on:
        counter+=1
        print("iteration: {}".format(counter))
        if( counter >stopafter):
            print("maximum message read({}) stop clean loader".format(stopafter))
            break
        try:
            msg = customercleanloader.poll(1.0)

            if msg is None:
                continue
            elif msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print(msg.error())
                    break
            else: 
                counter-=1
                print('Received message on clean topic: key {}'.format(msg.key().decode('utf-8')))
                SetLastDate(msg)
                go_on = False
                if msg.value().decode('utf-8') == "STOP":
                    go_on = False
        except KeyboardInterrupt:
            print("stopped by keyboard ")
            break
        except AttributeError:
            print("AttributeError ignore message. Probably a deleted messsage")
            continue
            
    customercleanloader.close()

In [9]:
##test
#print("before")
#for x in lastMeasurementsDict:
#    print(x)
#loadCleanConsumer()
#loadCleanData()
#print("after ")
#for x in lastMeasurementsDict:
#    print(x)
#

In [10]:
p = Producer({'bootstrap.servers':bootstrapservers })

In [11]:
def WritePM10(msg):
    keyMsg = msg.key().decode('utf-8')
    measurements = json.loads(msg.value().decode('utf-8'))
    print ('WritePM10')
    print ('key: '+keyMsg)

    global lastMeasurementsDict
    lastRead=''
    if(keyMsg in lastMeasurementsDict):
        lastRead = lastMeasurementsDict[keyMsg]
    data=[]
    for m in sorted(measurements,key=lambda m: getDatetimefromString( m['createdAt'])):      
        isNewValue= not lastRead or (
           getDatetimefromString( lastRead['createdAt'])  <
            getDatetimefromString( m['createdAt']))
        if(isNewValue):
          #  printTemperatureFromBox(s)
            #print(m["value"])
            lastRead = m                
   
            # Asynchronously produce a message, the delivery report callback
            # will be triggered from poll() above, or flush() below, when the message has
            # been successfully delivered or failed permanently.
            data.append(m)
            
    print("found {} new measurements".format(len(data)))
    if (len(data)>0) :       
        jsonData = json.dumps(data)       
        p.poll(0) 
        p.produce("clean-latest", key=keyMsg, value = jsonData.encode('utf-8'), callback=delivery_report)

    # Wait for any outstanding messages to be delivered and delivery report
    # callbacks to be triggered.
    p.flush()   


In [12]:
#single run for testing
# todo fix avro schema
def SingleRun():
    go_on = True

    while go_on:
        msg = cRaw.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

        print('Received message: {}'.format(msg.key().decode('utf-8')))
        WritePM10(msg)
        go_on = False
        if msg.value().decode('utf-8') == "STOP":
            go_on = False

    cRaw.close()



In [15]:
from IPython.display import clear_output
go_on = True
while go_on:
    clear_output(wait=True)
    loadCleanConsumer()
    loadCleanData()
    LoadRawConsumer()
    SingleRun()

loading new consumer
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
maximum message read(5) stop clean loader
loading new consumer


KeyboardInterrupt: 